# Group By, Having & Count

In [1]:
# import package
from google.cloud import bigquery

## Example: Which Hacker News comments generated the most discussion?

 The Hacker News dataset contains informaiton on stories and comments from the Hacker News social networking site. 
 
 Using information from <code>comments</code> table, let's answer the question above.

In [10]:
# [reference] how "comments" table looks like

# create client object
client = bigquery.Client()

# construct a reference to the dataset 
dataset_ref = client.dataset("hacker_news", project = "bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# construct a reference to the table, "comments"
table_ref = dataset_ref.table("comments")

# API request - fetche the table
comments = client.get_table(table_ref)

# preview the table
client.list_rows(comments, max_results = 5).to_dataframe()

/Users/haneul/opt/anaconda3/lib/python3.7/site-packages/google/cloud/bigquery/table.py:1763: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the REST endpoint.
  if not self._validate_bqstorage(bqstorage_client, create_bqstorage_client):


,id,by,author,time,time_ts,text,parent,deleted,dead,ranking
0,2701393,5l,5l,1309184881,2011-06-27 14:28:01+00:00,And the glazier who fixed all the broken windo...,2701243,None,None,0
1,5811403,99,99,1370234048,2013-06-03 04:34:08+00:00,Does canada have the equivalent of H1B/Green c...,5804452,None,None,0
2,21623,AF,AF,1178992400,2007-05-12 17:53:20+00:00,"Speaking of Rails, there are other options in ...",21611,None,None,0
3,10159727,EA,EA,1441206574,2015-09-02 15:09:34+00:00,Humans and large livestock (and maybe even pet...,10159396,None,None,0
4,2988424,Iv,Iv,1315853580,2011-09-12 18:53:00+00:00,I must say I reacted in the same way when I re...,2988179,None,None,0


In [11]:
# check the table schema
comments.schema

[SchemaField('id', 'INTEGER', 'NULLABLE', 'Unique comment ID', (), None),
 SchemaField('by', 'STRING', 'NULLABLE', 'Username of commenter', (), None),
 SchemaField('author', 'STRING', 'NULLABLE', 'Username of author', (), None),
 SchemaField('time', 'INTEGER', 'NULLABLE', 'Unix time', (), None),
 SchemaField('time_ts', 'TIMESTAMP', 'NULLABLE', 'Human readable time in UTC (format: YYYY-MM-DD hh:mm:ss)', (), None),
 SchemaField('text', 'STRING', 'NULLABLE', 'Comment text', (), None),
 SchemaField('parent', 'INTEGER', 'NULLABLE', 'Parent comment ID', (), None),
 SchemaField('deleted', 'BOOLEAN', 'NULLABLE', 'Is deleted?', (), None),
 SchemaField('dead', 'BOOLEAN', 'NULLABLE', 'Is dead?', (), None),
 SchemaField('ranking', 'INTEGER', 'NULLABLE', 'Comment ranking', (), None)]

<br>
According to the column descriptions from the table schema, 
<ul>
    <li><code>parent</code> column indicates the comment that was replied to</li>
    <li> the <code>id</code> column has the unique ID used to identify each comment.</li>
</ul>

Therefore, I picked these two column to get the answer to our question.

Now, I can **GROUP BY** the parent column, and **COUNT()** the id column to figure out the number of comments that were made as responses to a specific comment.
Since the question is about finding pouplar comments, I'll look at comments with more than ten replies. For this, I'll add **HAVING** after **GROUP BY**, so that the result is to be limited to the comments with more than 10 replies.

In [13]:
# query to select comments that received more than 10 replies
query="""
      SELECT parent, COUNT(1) AS NumPosts
      FROM `bigquery-public-data.hacker_news.comments`
      GROUP BY parent
      HAVING COUNT(1) > 10
      ORDER BY NumPosts DESC
      """

# set up the query (cancel the query if it would use too much of
# your quota, with the limit set to 10GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed = 10**10)
queryjob = client.query(query, job_config = safe_config)

# API request - run the query,and convert the results to a pandas dataframe
popular_comments = queryjob.to_dataframe()

# view the first 5 rows of the dataframe
popular_comments.head()

,parent,NumPosts
0,363,1311
1,9812245,902
2,9996333,850
3,9303396,785
4,10152809,733


## Supplement : Aliasing and COUNT(1)

**Aliasing**: As my query above, "AS NumPosts" were added to "COUNT(id)". It is to give label name on result. You can see the 2nd column name is set as such. It is called "Aliasing"

**COUNT(1)**: It counts the rows in each group. It comes handy when you are unsure what to put inside the parentheses. It also scans less data than if supplied column names <font color="green">(making it faster and using less of your data assess quota.)</font>

**Note on using GROUP BY**: All variables must be passed to either a 
1. GROUP BY command or
2. an aggregation function 

The following query won't work because <code>author</code> isn't passed to an aggregate function or a GROUP BY clause:
<br>

query_bad = """ <br>
            SELECT author, parent, COUNT(id)<br>
            FROM `bigquery-public-data.hacker_news.comments` <br>
            GROUP BY parent <br>
            """ <br>